In [1]:
from PatchGeneration.Modules.Mesh import *

In [2]:
mesh = Mesh.readFile('./PatchGeneration/Object/example_object.obj')

In [3]:
vertices = mesh.getVertices()
print(vertices.shape)

(22335, 3)


In [4]:
def rowwise_unique(x):
    unique = np.unique(x)
    correct_size = np.empty(x.shape)
    correct_size.fill(np.nan)
    correct_size[:len(unique)] = unique
    return correct_size

example = np.array([[1, 3, 2], [1, 1, 3]])
example2 = np.array([[1, 3, 4], [1, 1, 5]])
unique = np.apply_along_axis(rowwise_unique, 1, example)
union = np.apply_along_axis(rowwise_unique, 1, np.concatenate((example, example2), axis=1))
print(f"Row-wise unique: {unique}")
print(f"Row-wise union: {union}")
print(union[:,~np.all(np.isnan(union), axis=0)])


Row-wise unique: [[ 1.  2.  3.]
 [ 1.  3. nan]]
Row-wise union: [[ 1.  2.  3.  4. nan nan]
 [ 1.  3.  5. nan nan nan]]
[[ 1.  2.  3.  4.]
 [ 1.  3.  5. nan]]


In [5]:
ring = 2
selfk = 4

# (face_index, edge_neighbor_index) -> face_index
f2f = mesh.f2f

# (face_index, ring_neighbor_index) -> face_index
result = np.arange(len(mesh.f)).reshape((len(mesh.f), 1))
for _ in range(ring):
    # (face_index, found_neighbors_index) -> face_index
    nb = f2f[result.astype(int)].reshape((len(mesh.f), -1))
    concat = np.concatenate((result, nb), axis=1)
    unique = np.apply_along_axis(rowwise_unique, 1, concat)
    result = unique[:, ~np.all(np.isnan(unique), axis=0)]

mask = np.isnan(result)
result[mask] = 0
nhs = np.ma.array(result, mask=mask)
areas = mesh.getAreas(nhs)
avg_tworing_area = np.sum(areas, axis=1) / (nhs.shape[1] - nhs.mask.sum(1))
r = selfk * avg_tworing_area ** 0.5
centers = np.average(mesh.getVertices()[mesh.f], axis=-2)

def getFacesInRanges(mesh, centers, ranges):
    translated = mesh.getVertices()[None] - centers[:, None]
    dist = np.linalg.norm(translated, axis=-1)
    vm = dist < ranges[:, None]
    print(vm.shape)
    vertices = np.arange(vm.shape[0])[vm]
    faces = mesh.getTrianglesOfVertices(vertices)
    return faces

faces_in_range = getFacesInRanges(mesh, centers, r)
MAX_QUEUE_LENGTH = 128

# (vertex_index, coordinate_index -> coordinate_value)
v = mesh.getVertices()
center_face_index = 0 # Trivial
 # (vertex_index, coordinate_index -> coordinate_value)[(face_index, corner_index -> vertex_index)] =
 # (face_index, corner_index, coordinate_index -> coordinate_value)
centers = v[mesh.f]
# (face_index, element_index) -> face_index
faces_in_range = np.full((len(mesh.f), MAX_QUEUE_LENGTH), np.nan)
faces_in_range[:, 0] = np.arange(len(mesh.f))

# (face_index, element_index) -> face_index
faces_to_check = faces_in_range.copy()
while np.isnan(faces_to_check).argmax(axis=1).sum() > 0:
    # Pop faces_to_check
    current_faces_to_check = faces_to_check[:, 0].astype(int)
    faces_to_check[:, :-1] = faces_to_check[:, 1:]
    faces_to_check[:, -1] = np.nan
    # (face_index, neighbor_index) -> face_index[(face_index) -> face_index]
    # (face_index, neighbor_index) -> face_index
    adjacent_faces = mesh.f2f[current_faces_to_check]
    # Remove faces that are already checked / in range
    mark_to_remove = (adjacent_faces[..., None] == faces_in_range[:, None]).sum(-1).astype(bool)
    new_adjacent_faces = np.ma.array(adjacent_faces, mask=mark_to_remove)
    # Remove faces that are out of range
    # indexing: (vertex_index, coordinate_index -> coordinate_value)[(face_index, corner_index -> vertex_index)[(face_index, neighbor_index -> face_index)]] =
    # (vertex_index, coordinate_index -> coordinate_value)[face_index, neighbor_index, corner_index -> vertex_index] = 
    # (face_index, neighbor_index, corner_index, coordinate_index -> coordinate_value)
    # centers: (face_index, corner_index, coordinate_index -> coordinate_value)
    # dist: (center_face_index, neighbor_index, corner_index, coordinate_index -> coordinate_value)
    dist = np.linalg.norm(v[mesh.f[new_adjacent_faces]] - centers[:, None], axis=3)
    # faces_to_append = np.delete(unique_adjacent_faces, np.where(np.any(dist > range, axis=1)))
    # faces_in_range = np.append(faces_in_range, faces_to_append)
    # faces_to_check = faces_to_append
# patch = self.mesh.createPatch(faces_in_range, fi)
# return patch

In [ ]:
_v = np.random.rand(10, 3)
_c = np.random.rand(20, 3)
(_v[None]-_c[:, None]).shape


(20, 10, 3)